In [2]:
# Documentation: https://www.crummy.com/software/BeautifulSoup/bs4/doc/

from bs4 import BeautifulSoup
import requests

page = requests.get("https://www.starwars.com/news/star-wars-the-clone-wars-chronological-episodeorder/")

soup = BeautifulSoup(page.content)

# Print the first 950 chars of the page.
print(soup.prettify()[:950])



<!DOCTYPE html>
<!--[if lt IE 7]> <html class="no-js lt-ie9 lt-ie8 lt-ie7" lang="en-US"> <![endif]-->
<!--[if IE 7]>    <html class="no-js lt-ie9 lt-ie8" lang="en-US"> <![endif]-->
<!--[if IE 8]>    <html class="no-js lt-ie9" lang="en-US"> <![endif]-->
<!--[if gt IE 8]><!-->
<html lang="en-US">
 <!--<![endif]-->
 <head>
  <meta charset="utf-8"/>
  <!-- Google Chrome Frame for IE -->
  <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
  <!-- mobile meta -->
  <meta content="True" name="HandheldFriendly"/>
  <meta content="320" name="MobileOptimized"/>
  <meta content="width=device-width, initial-scale=1.0" name="viewport"/>
  <!-- icons & favicons -->
  <meta content="Star Wars" name="apple-mobile-web-app-title"/>
  <link href="/apple-touch-icon.png" rel="apple-touch-icon"/>
  <link href="/apple-touch-icon.png" rel="icon" sizes="196x196"/>
  <link href="//a.dilcdn.com/sw/favicon-44660648232d.ico" rel="shortcut icon"/>
  <me


In [3]:

print(soup.title.string)


Star Wars: The Clone Wars Chronological Episode Order | StarWars.com


In [15]:

def begins_with_number(x):
    numbers = ["1","2","3","4","5","6","7","8","9","10"];
    return x[0] in numbers


def parse_detailed_info_page(url):
    
    #     request the page
    detail_page = requests.get(episode_detail_page_link)
    detail_soup = BeautifulSoup(detail_page.content)

#     description div has a quote and a description.
    description_div = detail_soup.find('div', class_="desc")
    
    
    start_quote_span = description_div.find('span', 'quote')
    if(start_quote_span):
        start_quote = start_quote_span.text
    else:
        start_quote = ""
        
    description = ""
    
#   first child is empty
#   2nd child is the quote
#   3rd child (index 2) is the description text. but it has some leading white-space.
    for index, child in enumerate(description_div.children):
        if(index==2):
            description = child.strip() 
            
    return (start_quote, str(description))



headers = ["ep_number", "episode_name", "episode_detail_page_link", "start_quote", "description"]

all_episodes = []

# for row in soup.find_all('td', class_="xl65")[0:3]:
for row in soup.find_all('td', class_="xl65"):
    if(row.string and begins_with_number(row.string)):
        ep_number = row.string
        a_link = row.parent.find("a")
        episode_name = a_link.text
        episode_detail_page_link = a_link['href']
        
        detail_page = requests.get(episode_detail_page_link)
        detail_soup = BeautifulSoup(detail_page.content)
        
        start_quote, description = parse_detailed_info_page(episode_detail_page_link)
                
        print(ep_number)
        print(episode_name)
        print(episode_detail_page_link)
        print(start_quote)
        print(description)

        episode = [ep_number, episode_name, episode_detail_page_link, start_quote, description]
        all_episodes.append(episode)
        print(" ")

    




216
Cat and Mouse
https://www.starwars.com/tv-shows/clone-wars/cat-and-mouse-episode-guide
"A wise leader knows when to follow"
A Separatist blockade surrounds Christophsis. Anakin's fleet is tasked with deploying relief supplies to the surface but they are massively out-gunned and out-maneuvered by the skillful Admiral Trench. Obi-Wan arrives and unveils the Republic's new weapon: a stealth ship. The stealth ship is the last hope at besting the cunning Separatist fleet commander and aiding the battered people of Christophsis.
 
116
Hidden Enemy
https://www.starwars.com/tv-shows/clone-wars/the-hidden-enemy-episode-guide
"Truth enlightens the mind, but won't always bring happiness to your heart."
Anakin Skywalker and Obi-Wan Kenobi lead Republic forces in an attempt to defeat the droid armies and free the planet of Christophsis from the Separatist siege. When the Republic is ambushed and forced to retreat, it becomes clear that someone in their midst has set them up. The Jedi believe th

114
Defenders of Peace
https://www.starwars.com/tv-shows/clone-wars/defenders-of-peace-episode-guide
"When surrounded by war, one must eventually choose a side."
The leader of the Lurmen, Tee Watt Kaa, refuses to shelter the Jedi from arriving Separatist forces, fearing that such an action would unnecessarily bring his neutral people into the war. Ahsoka, Aayla, the wounded Anakin, and the rest of the clones respect their wishes as they don't want to jeopardize the village with their presence, so they have to retreat into the wilds of the planet. Separatist Neimoidian General Lok Durd arrives to test a devastating weapon, which they plan to use on the Lurmen! Ultimately, the villagers have to decide whether they will lay down to the Separatists or fight with the Jedi to defend their village.
 
115
Trespass
https://www.starwars.com/tv-shows/clone-wars/trespass-episode-guide
"Arrogance diminishes wisdom."
Arriving on the desolate ice world of Orto Plutonia, Anakin Skywalker and Obi-Wan K

204
Senate Spy
https://www.starwars.com/tv-shows/clone-wars/senate-spy-episode-guide
"A true heart should never be doubted."
The Jedi Council suspects that Senator Rush Clovis, an InterGalactic Banking Clan delegate and former colleague of Padmé Amidala, may be working for the Separatists. Against Anakin's wishes, the Council asks Padmé if she would accompany Rush to Cato Neimoidia to uncover any clues regarding the Senator's true allegiances. Padmé agrees, feeling that her past relationship with the Senator -- which for a time had been romantic until she ended it -- might give her an advantage in the investigation. Fending off Anakin's jealousy, as well as an attempt on her life by Neimoidian Lott Dod, Padmé successfully reveals Clovis to be conspiring with the Separatists to build a military droid factory on Geonosis.
 
205
Landing at Point Rain
https://www.starwars.com/tv-shows/clone-wars/landing-at-point-rain-episode-guide
"Believe in yourself or no one else will."
Anakin, Obi-wan,

306
The Academy
https://www.starwars.com/tv-shows/clone-wars/the-academy-episode-guide
"Those who enforce the law must obey the law."
Ahsoka is assigned to teach a class at a leadership academy on Mandalore. Soon after she arrives, Duchess Satine's zealous nephew -- Korkie -- and his classmates uncover a nefarious plot. Prime Minister Almec is revealed to be an active part of the black market conspiracy on Mandalore, and he attempts to permanently silence Duchess Satine and the cadets before they expose his corruption. Ahsoka and the cadets defeat Almec before he can succeed.
 
307
Assassin
https://www.starwars.com/tv-shows/clone-wars/assassin-episode-guide
"The future has many paths -- choose wisely."
Tasked to protect Senator Padmé Amidala during a political mission to Alderaan, Padawan Ahsoka Tano is plagued by recurring visions of recently-deceased bounty hunter, Aurra Sing. Unable to clear her mind, Ahsoka worries that the stress of conflict is finally getting to her until her vis

314
Witches of the Mist
https://www.starwars.com/tv-shows/clone-wars/witches-of-the-mist-episode-guide
"The path to evil may bring great power, but not loyalty."
Anakin and Obi-Wan, sent to track down the mysterious figure behind the deaths of several Jedi, soon find themselves on the trail of the monstrous apprentice that Ventress has created: Savage Opress, who has been trained in the ways of the Sith by Dooku.     When Opress returns to Count Dooku after an unsuccessful assignment, Ventress strikes. With Opress at her side, she attacks Dooku. Opress' loyalty is fleeting, however, and Dooku and Ventress discover that this new apprentice has a will of his own.     When Savage returns to Mother Talzin, she gives him a new task: To seek the Outer Rim for his long-lost brother....
 
315
Overlords
https://www.starwars.com/tv-shows/clone-wars/overlords-episode-guide
"Balance is found in the one who faces his guilt." 
A mysterious force draws Anakin, Obi-Wan and Ahsoka to a distant planet, 

405
Mercy Mission
https://www.starwars.com/tv-shows/clone-wars/mercy-mission-episode-guide
"Understanding is honoring the truth beneath the surface."
After groundquakes have devastated the planet Aleen, a Republic relief effort arrives, including the droids R2-D2 and C-3PO. When the native Aleena's pleas go unheard, it becomes the duty of the droids to embark on a surreal journey through Aleen's subterranean world to save the planet.
 
406
Nomad Droids
https://www.starwars.com/tv-shows/clone-wars/nomad-droids-episode-guide
"Who's the more foolish, the fool or the fool who follows him?"
C-3PO and R2-D2's bizarre caper continues. Forced to escape a Separatist attack in a Y-wing fighter, the droids visit the world of the Patitites, the odd planet Balnab, and the inside of a pirate warship where they are forced to fight in a gladiatorial arena.
 
407
Darkness on Umbara
https://www.starwars.com/tv-shows/clone-wars/darkness-on-umbara-episode-guide
"The first step toward loyalty is trust."
Wh

KeyboardInterrupt: 

In [12]:
import pandas as pd

df = pd.DataFrame(all_episodes, columns=headers)
df.head()


,ep_number,episode_name,episode_detail_page_link,start_quote,description
0,216,Cat and Mouse,https://www.starwars.com/tv-shows/clone-wars/c...,"""A wise leader knows when to follow""",A Separatist blockade surrounds Christophsis. ...
1,116,Hidden Enemy,https://www.starwars.com/tv-shows/clone-wars/t...,"""Truth enlightens the mind, but won't always b...",Anakin Skywalker and Obi-Wan Kenobi lead Repub...
2,301,Clone Cadets,https://www.starwars.com/tv-shows/clone-wars/c...,"""Brothers in arms are brothers for life.""",Five headstrong clones struggle to complete th...
3,303,Supply Lines,https://www.starwars.com/tv-shows/clone-wars/s...,"""Where there's a will, there's a way.""","Ryloth is under siege. Trapped on the surface,..."
4,101,Ambush,https://www.starwars.com/tv-shows/clone-wars/a...,"""Great leaders inspire greatness in others.""",Jedi Master Yoda and three clone troopers -- J...


> An attack by a devastating new Separatist weapon -- the powerful starship Malevolence -- leaves Jedi Master Plo Koon and his clone troopers struggling to survive until Anakin and Ahsoka can find them.
